<a href="https://colab.research.google.com/github/ZS4MLDL/learn_pytorch/blob/main/04_Gans_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import time

In [2]:
os.makedirs('./images/gan', exist_ok=True)

In [3]:
BATCH_SIZE = 64
N_EPOCHS = 100
IMAGE_SIZE = 28 * 28
LATENT_DIM = 100
PRINT_EVERY = 5
N_SHOW = 5

device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
transforms = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
])

In [6]:
data = datasets.MNIST('.data', train=True, download=True,transform=transforms)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 15.8MB/s]


Extracting .data/MNIST/raw/train-images-idx3-ubyte.gz to .data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 490kB/s]


Extracting .data/MNIST/raw/train-labels-idx1-ubyte.gz to .data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.40MB/s]


Extracting .data/MNIST/raw/t10k-images-idx3-ubyte.gz to .data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 8.03MB/s]

Extracting .data/MNIST/raw/t10k-labels-idx1-ubyte.gz to .data/MNIST/raw



In [7]:
data

Dataset MNIST
    Number of datapoints: 60000
    Root location: .data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [8]:
iterator = DataLoader(data,BATCH_SIZE,shuffle=True,drop_last=True)

In [10]:
class Generator(nn.Module):
  def __init__(self, latent_dim, image_size):
    super().__init__()
    self.main = nn.Sequential(
        nn.Linear(latent_dim, 256),
        nn.LeakyReLU(0.2),

        nn.Linear(256, 512),
        nn.LeakyReLU(0.2),

        nn.Linear(512, 1024),
        nn.LeakyReLU(0.2),

        nn.Linear(1024, image_size),
        nn.Tanh()

        )

    def forward(self, x):
      return self.main(x)


In [ ]:
class Discriminator(nn.Module):
  def __init__(self,image_size):
    super().__init__()
    self.main = nn.Sequential(
        nn.Linear(784, 1024),
        nn.LeakyReLU(0.2),
        nn.Dropout(0.3)

        nn.Linear(1024, 512),
        nn.LeakyReLU(0.2),
        nn.Dropout(0.3)

        nn.Linear(512, 256)
        nn.LeakyReLU(0.2)
        nn.Dropout(0.3)
    )